# Class Notes

## Bijectors Intro

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [3]:
z = tf.constant([1., 2., 3.])
scale = tfb.Scale(2.)

x = scale.forward(z)
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 6.], dtype=float32)>

In [4]:
scale.inverse(tf.constant([5., 3., 1.])) # ie x / scale

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.5, 1.5, 0.5], dtype=float32)>

### Shift

In [9]:
z = tf.constant([1., 2., 3.])
scale = tfb.Scale(2.)
shift = tfb.Shift(1.)

scale_and_shift = tfb.Chain([shift, scale])
scale_and_shift

In [10]:
scale_and_shift.forward(z)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3., 5., 7.], dtype=float32)>

In [13]:
scale_and_shift.inverse(z) # Subtract and Scale(divide)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0. , 0.5, 1. ], dtype=float32)>

### Transform Random Variables

In [14]:
normal = tfd.Normal(loc=0, scale=1.)
z = normal.sample(3)
z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.18415736,  1.0752617 ,  0.15316185], dtype=float32)>

In [15]:
scale_and_shift = tfb.Chain([tfb.Shift(1.), tfb.Scale(2.)])
scale_and_shift

In [16]:
x = scale_and_shift.forward(z)
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.63168526, 3.1505234 , 1.3063238 ], dtype=float32)>

In [19]:
log_prob_z = normal.log_prob(z)
log_prob_z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.9358955, -1.4970324, -0.9306678], dtype=float32)>

In [21]:
log_prob_x = normal.log_prob(z) - scale_and_shift.forward_log_det_jacobian(z, event_ndims=0)
log_prob_x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.6290426, -2.1901796, -1.6238151], dtype=float32)>